In [2]:
# Load the caret package
library(caret)

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


# caret Package

### A Short Introduction to the caret Package

The caret package (short for Classification And REgression Training) contains functions to streamline the model training process for complex regression and classification problems. The package utilizes a number of R packages but tries not to load them all at package start-up (by removing formal package dependencies, the package startup time can be greatly decreased). The package “suggests” field includes 32 packages. caret loads packages as needed and assumes that they are installed. If a modeling package is missing, there is a prompt to install it.

### caret has several functions that attempt to streamline the model building and evaluation process, as well as feature selection and other techniques.

One of the primary tools in the package is the train function which can be used to

evaluate, using resampling, the effect of model tuning parameters on performance
choose the ``optimal’’ model across these parameters
estimate model performance from a training set


## split the data
a training set and a test set. To do this, the createDataPartition function is used:

By default, createDataPartition does a stratified random split of the data. To partition the data:

In [ ]:
inTrain <- createDataPartition(
  y = Sonar$Class,
  ## the outcome data are needed
  p = .75,
  ## The percentage of data in the
  ## training set
  list = FALSE
)

# another shape for split data

### createDataPartition()

takes as input the Y variable in the source dataset and the percentage data that should go into training as the p argument. It returns the rownumbers that should form the training dataset.

Plus, you need to set list=F, to prevent returning the result as a list.

In [ ]:
# Create the training and test datasets
set.seed(100)

# Step 1: Get row numbers for the training data
trainRowNumbers <- createDataPartition(orange$Purchase, p=0.8, list=FALSE)

# Step 2: Create the training  dataset
trainData <- orange[trainRowNumbers,]

# Step 3: Create the test dataset
testData <- orange[-trainRowNumbers,]

# Store X and Y for later use.
x = trainData[, 2:18]
y = trainData$Purchase

## Descriptive statistics

Before moving to missing value imputation and feature preprocessing, let’s observe the descriptive statistics of each column in the training dataset.

The skimr package provides a nice solution to show key descriptive stats for each column.

The skimr::skim_to_wide() produces a nice dataframe containing the descriptive stats of each of the columns. The dataframe output includes a nice histogram drawn without any plotting help

In [ ]:
library(skimr)
skimmed <- skim_to_wide(trainData)
skimmed[, c(1:5, 9:11, 13, 15:16)]

# How to impute missing values using preProcess()?

We’ve seen that the dataset has few missing values across all columns, we may to do well to impute it. Impute, means to fill it up with some meaningful values.


If the feature is a continuous variable, it is a common practice to replace the missing values with the mean of the column. And if it’s a categorical variable, replace the missings with the most frequently occurring value, aka, the mode.

But this is quite a basic and a rather rudimentary approach.

Instead what can be done is, you can actually predict the missing values by considering the rest of the available variables as predictors. A popular algorithm to do imputation is the k-Nearest Neighbors. This can be quickly and easily be done using caret.

### To predict the missing values with k-Nearest Neighbors using preProcess():

1-You need to set the method=knnImpute for k-Nearest Neighbors and apply it on the training data. This creates a preprocess model.

2-Then use predict() on the created preprocess model by setting the newdata argument on the same training data.

Caret also provides bagImpute as an alternative imputation algorithm.

In [ ]:
#               first step
# Create the knn imputation model on the training data
preProcess_missingdata_model <- preProcess(trainData, method='knnImpute')
preProcess_missingdata_model

In [ ]:
#         seond step
# Use the imputation model to predict the values of missing data points
library(RANN)  # required for knnInpute
trainData <- predict(preProcess_missingdata_model, newdata = trainData)
anyNA(trainData)

# How to create One-Hot Encoding (dummy variables)?

Let me first explain what is one-hot encoding and why it is required.

Suppose if you have a categorical column as one of the features, it needs to be converted to numeric in order for it to be used by the machine learning algorithms.

In caret, one-hot-encodings can be created using dummyVars(). Just pass in all the features to dummyVars() as the training data and all the factor columns will automatically be converted to one-hot-encodings.


In [ ]:
# One-Hot Encoding
# Creating dummy variables is converting a categorical variable to as many binary variables as here are categories.
dummies_model <- dummyVars(Purchase ~ ., data=trainData)

# Create the dummy variables using predict. The Y variable (Purchase) will not be present in trainData_mat.
trainData_mat <- predict(dummies_model, newdata = trainData)

# # Convert to dataframe
trainData <- data.frame(trainData_mat)

# # See the structure of the new dataset
str(trainData)

In above case, we had one categorical variable, Store7 with 2 categories. It was one-hot-encoded to produce two new columns – Store7.No and Store7.Yes.

# How to preprocess to transform the data?
With the missing values handled and the factors one-hot-encoded, our training dataset is now ready to undergo variable transformations if required.

So what type of preprocessing are available in caret?

### range:
Normalize values so it ranges between 0 and 1

### center: 
Subtract Mean

### scale:
Divide by standard deviation

### BoxCox: 
Remove skewness leading to normality. Values must be > 0

### YeoJohnson:
Like BoxCox, but works for negative values.

### expoTrans:
Exponential transformation, works for negative values.

### pca:
Replace with principal components

### ica: 
Replace with independent components

### spatialSign:
Project the data to a unit circle


For our problem, let’s convert all the numeric variables to range between 0 and 1, by setting method=range in preProcess().

In [ ]:
preProcess_range_model <- preProcess(trainData, method='range')
trainData <- predict(preProcess_range_model, newdata = trainData)

# Append the Y variable
trainData$Purchase <- y

apply(trainData[, 1:10], 2, FUN=function(x){c('min'=min(x), 'max'=max(x))})

## How to visualize the importance of variables using featurePlot()

Now that the preprocessing is complete, let’s visually examine how the predictors influence the Y (Purchase).

In this problem, the X variables are numeric whereas the Y is categorical.

So how to gauge if a given X is an important predictor of Y?

A simple common sense approach is, if you group the X variable by the categories of Y, a significant mean shift amongst the X’s groups is a strong indicator (if not the only indicator) that X will have a significant role to help predict Y.

It is possible to watch this shift visually using box plots and density plots.

In fact, caret’s featurePlot() function makes it so convenient.

Simply set the X and Y parameters and set plot='box'. You can additionally adjust the label font size (using strip) and the scales to be free as I have done in the below plot.

In [ ]:
#  strip and scale are additional 
featurePlot(x = trainData[, 1:18], 
            y = trainData$Purchase, 
            plot = "box",
            strip=strip.custom(par.strip.text=list(cex=.7)),
            scales = list(x = list(relation="free"), 
                          y = list(relation="free")))

In [ ]:
featurePlot(x = trainData[, 1:18], 
            y = trainData$Purchase, 
            plot = "density",
            strip=strip.custom(par.strip.text=list(cex=.7)),
            scales = list(x = list(relation="free"), 
                          y = list(relation="free")))


# correlation 

In [ ]:
library(corrplot)
M <- cor(numeric_mydata)
M
corrplot(M, ,method='square',type='upper')

In [ ]:
#Finding how many correlations are bigger than 0.70
k = 0
for(i in 1:25){
for(r in 1:25){
  if(M[i,r]> 0.70 & i != r){
    print(rownames(M)[i])
    print(colnames(M)[r])
    print(M[i,r])
    k= k + 1
  }
}  }
print(k/2)

options( warn = -1 )

# How to do feature selection using recursive feature elimination (rfe)?

Most machine learning algorithms are able to determine what features are important to predict the Y. But in some scenarios, you might be need to be careful to include only variables that may be significantly important and makes strong business sense.

This is quite common in banking, economics and financial institutions.

Or you might just be doing an exploratory analysis to determine important predictors and report it as a metric in your analytics dashboard.

Or if you are using a traditional algorithm like like linear or logistic regression, determining what variable to feed to the model is in the hands of the practitioner.

Given such requirements, you might need a rigorous way to determine the important variables first before feeding them to the ML algorithm.

A good choice of selecting the important features is the recursive feature elimination (RFE).

# RFE works in 3 broad steps:

Step 1: Build a ML model on a training dataset and estimate the feature importances on the test dataset.

Step 2: Keeping priority to the most important variables, iterate through by building models of given subset sizes, that is, subgroups of most important predictors determined from step 1. Ranking of the predictors is recalculated in each iteration.

Step 3: The model performances are compared across different subset sizes to arrive at the optimal number and list of final predictors.

It can be implemented using the rfe() function and you have the flexibility to control what algorithm rfe uses and how it cross validates by defining the rfeControl().


In [ ]:

ctrl <- rfeControl(functions = rfFuncs,
                   method = "repeatedcv",
                   repeats = 5,
                   verbose = FALSE)


In [ ]:
subsets <- c(1:5, 10, 15, 18)

lmProfile <- rfe(x=trainData[, 1:18], y=trainData$Purchase,
                 sizes = subsets,
                 rfeControl = ctrl)

lmProfile

n the above code, we call the rfe() which implements the recursive feature elimination.

Apart from the x and y datasets, RFE also takes two important parameters.

sizes
rfeControl
The sizes determines what all model sizes (the number of most important features) the rfe should consider. In above case, it iterates models of size 1 to 5, 10, 15 and 18.

The rfeControl parameter on the other hand receives the output of the rfeControl() as values. If you look at the call to rfeControl() we set what type of algorithm and what cross validation method should be used.

In above case, the cross validation method is repeatedcv which implements k-Fold cross validation repeated 5 times, which is rigorous enough for our case.

Once rfe() is run, the output shows the accuracy and kappa (and their standard deviation) for the different model sizes we provided. The final selected model subset size is marked with a * in the rightmost Selected column.

From the above output, a model size of 3 with LoyalCH, PriceDiff and StoreID seems to achieve the optimal accuracy.

# hi

That means, out of 18 other features, a model with just 3 features outperformed many other larger model. Interesting isn’t it! Can you explain why?

However, it is not a mandate that only including these 3 variables will always give high accuracy over larger sized models.

Thats because, the rfe() we just implemented is particular to random forest based rfFuncs

# n the next step, we will build the actual randomForest model on trainData

## How to train() the model and interpret the results?

In [3]:
#To know what models caret supports, run the following:

# See available algorithms in caret
modelnames <- paste(names(getModelInfo()), collapse=',  ')
modelnames

[1] "ada,  AdaBag,  AdaBoost.M1,  adaboost,  amdai,  ANFIS,  avNNet,  awnb,  awtan,  bag,  bagEarth,  bagEarthGCV,  bagFDA,  bagFDAGCV,  bam,  bartMachine,  bayesglm,  binda,  blackboost,  blasso,  blassoAveraged,  bridge,  brnn,  BstLm,  bstSm,  bstTree,  C5.0,  C5.0Cost,  C5.0Rules,  C5.0Tree,  cforest,  chaid,  CSimca,  ctree,  ctree2,  cubist,  dda,  deepboost,  DENFIS,  dnn,  dwdLinear,  dwdPoly,  dwdRadial,  earth,  elm,  enet,  evtree,  extraTrees,  fda,  FH.GBML,  FIR.DM,  foba,  FRBCS.CHI,  FRBCS.W,  FS.HGD,  gam,  gamboost,  gamLoess,  gamSpline,  gaussprLinear,  gaussprPoly,  gaussprRadial,  gbm_h2o,  gbm,  gcvEarth,  GFS.FR.MOGUL,  GFS.LT.RS,  GFS.THRIFT,  glm.nb,  glm,  glmboost,  glmnet_h2o,  glmnet,  glmStepAIC,  gpls,  hda,  hdda,  hdrda,  HYFIS,  icr,  J48,  JRip,  kernelpls,  kknn,  knn,  krlsPoly,  krlsRadial,  lars,  lars2,  lasso,  lda,  lda2,  leapBackward,  leapForward,  leapSeq,  Linda,  lm,  lmStepAIC,  LMT,  loclda,  logicBag,  LogitBoost,  logreg,  lssvmLinear,  lssvmPoly,  lssvmRadial,  lvq,  M5,  M5Rules,  manb,  mda,  Mlda,  mlp,  mlpKerasDecay,  mlpKerasDecayCost,  mlpKerasDropout,  mlpKerasDropoutCost,  mlpML,  mlpSGD,  mlpWeightDecay,  mlpWeightDecayML,  monmlp,  msaenet,  multinom,  mxnet,  mxnetAdam,  naive_bayes,  nb,  nbDiscrete,  nbSearch,  neuralnet,  nnet,  nnls,  nodeHarvest,  null,  OneR,  ordinalNet,  ordinalRF,  ORFlog,  ORFpls,  ORFridge,  ORFsvm,  ownn,  pam,  parRF,  PART,  partDSA,  pcaNNet,  pcr,  pda,  pda2,  penalized,  PenalizedLDA,  plr,  pls,  plsRglm,  polr,  ppr,  PRIM,  protoclass,  qda,  QdaCov,  qrf,  qrnn,  randomGLM,  ranger,  rbf,  rbfDDA,  Rborist,  rda,  regLogistic,  relaxo,  rf,  rFerns,  RFlda,  rfRules,  ridge,  rlda,  rlm,  rmda,  rocc,  rotationForest,  rotationForestCp,  rpart,  rpart1SE,  rpart2,  rpartCost,  rpartScore,  rqlasso,  rqnc,  RRF,  RRFglobal,  rrlda,  RSimca,  rvmLinear,  rvmPoly,  rvmRadial,  SBC,  sda,  sdwd,  simpls,  SLAVE,  slda,  smda,  snn,  sparseLDA,  spikeslab,  spls,  stepLDA,  stepQDA,  superpc,  svmBoundrangeString,  svmExpoString,  svmLinear,  svmLinear2,  svmLinear3,  svmLinearWeights,  svmLinearWeights2,  svmPoly,  svmRadial,  svmRadialCost,  svmRadialSigma,  svmRadialWeights,  svmSpectrumString,  tan,  tanSearch,  treebag,  vbmpRadial,  vglmAdjCat,  vglmContRatio,  vglmCumulative,  widekernelpls,  WM,  wsrf,  xgbDART,  xgbLinear,  xgbTree,  xyf"

## Each of those is a machine learning algorithm caret supports.

Yes, it’s a huge list!

And if you want to know more details like the hyperparameters and if it can be used of regression or classification problem, then do a modelLookup(algo).

Once you have chosen an algorithm, building the model is fairly easy using the train() function.

Let’s train a Multivariate Adaptive Regression Splines (MARS) model by setting the method='earth'.

The MARS algorithm was named as ‘earth’ in R because of a possible trademark conflict with Salford Systems. May be a rumor. Or not.

In [4]:
modelLookup('earth')

model,parameter,label,forReg,forClass,probModel
earth,nprune,#Terms,TRUE,TRUE,TRUE
earth,degree,Product Degree,TRUE,TRUE,TRUE


In [ ]:
# Train the model using randomForest and predict on the training data itself.

model_mars = train(Purchase ~ ., data=trainData, method='earth')
fitted <- predict(model_mars)

# But you may ask how is using train() different from using the algorithm’s function directly?

The difference is, besides building the model train() does multiple other things like:

Cross validating the model

Tune the hyper parameters for optimal model performance

Choose the optimal model based on a given evaluation metric

Preprocess the predictors (what we did so far using preProcess())

The train function also accepts the arguments used by the algorithm specified in the method argument.

In [ ]:
model_mars

# Prepare the test dataset and predict

A default MARS model has been selected.

Now in order to use the model to predict on new data, the data has to be preprocessed and transformed just the way we did on the training data.

Thanks to caret, all the information required for pre-processing is stored in the respective preProcess model and dummyVar model.

# You need to pass the testData through these models in the same sequence

### preProcess_missingdata_model –> dummies_model –> preProcess_range_model

In [ ]:
# Step 1: Impute missing values 
testData2 <- predict(preProcess_missingdata_model, testData)  

# Step 2: Create one-hot encodings (dummy variables)
testData3 <- predict(dummies_model, testData2)

# Step 3: Transform the features to range between 0 and 1
testData4 <- predict(preProcess_range_model, testData3)

# View
head(testData4[, 1:10])

# Predict on testData
The test dataset is prepared. Let’s predict the Y.

In [ ]:
# Predict on testData
predicted <- predict(model_mars, testData4)
head(predicted)

## Confusion Matrix

The confusion matrix is a tabular representation to compare the predictions (data) vs the actuals (reference). By setting mode='everything' pretty much most classification evaluation metrics are computed.

In [ ]:
# Compute the confusion matrix
confusionMatrix(reference = testData$Purchase, data = predicted, mode='everything', positive='MM')


# How to do hyperparameter tuning to optimize the model for better performance?

There are two main ways to do hyper parameter tuning using the train():

Set the tuneLength

Define and set the tuneGrid

tuneLength corresponds to the number of unique values for the tuning parameters caret will consider while forming the hyper parameter combinations.

Caret will automatically determine the values each parameter should take.

Alternately, if you want to explicitly control what values should be considered for each parameter, then, you can define the tuneGrid and pass it to train().

# 7.1. Setting up the trainControl()

The train() function takes a trControl argument that accepts the output of trainControl().

Inside trainControl() you can control how the train() will:

Cross validation method to use.

How the results should be summarised using a summary function
Cross validation method can be one amongst:

‘boot’: Bootstrap sampling

‘boot632’: Bootstrap sampling with 63.2% bias correction applied

‘optimism_boot’: The optimism bootstrap estimator

‘boot_all’: All boot methods.

‘cv’: k-Fold cross validation

‘repeatedcv’: Repeated k-Fold cross validation

‘oob’: Out of Bag cross validation

‘LOOCV’: Leave one out cross validation

‘LGOCV’: Leave group out cross validation

The summaryFunction can be twoClassSummary if Y is binary class or multiClassSummary if the Y has more than 2 categories.

By settiung the classProbs=T the probability scores are generated instead of directly predicting the class based on a predetermined cutoff of 0.5.

In [ ]:
# Define the training control
fitControl <- trainControl(
    method = 'cv',                   # k-fold cross validation
    number = 5,                      # number of folds
    savePredictions = 'final',       # saves predictions for optimal tuning parameter
    classProbs = T,                  # should class probabilities be returned
    summaryFunction=twoClassSummary  # results summary function
) 

# 7.2 Hyper Parameter Tuning using tuneLength

Let’s take the train() function we used before, plus, additionally set the tuneLength, trControl and metric.

In [ ]:
# Step 1: Tune hyper parameters by setting tuneLength
set.seed(100)
model_mars2 = train(Purchase ~ ., data=trainData, method='earth', tuneLength = 5, metric='ROC', trControl = fitControl)
model_mars2

# Step 2: Predict on testData and Compute the confusion matrix
predicted2 <- predict(model_mars2, testData4)
confusionMatrix(reference = testData$Purchase, data = predicted2, mode='everything', positive='MM')


# 7.3. Hyper Parameter Tuning using tuneGrid

Alternately, you can set the tuneGrid instead of tuneLength

In [ ]:
# Step 1: Define the tuneGrid
marsGrid <-  expand.grid(nprune = c(2, 4, 6, 8, 10), 
                        degree = c(1, 2, 3))

# Step 2: Tune hyper parameters by setting tuneGrid
set.seed(100)
model_mars3 = train(Purchase ~ ., data=trainData, method='earth', metric='ROC', tuneGrid = marsGrid, trControl = fitControl)
model_mars3

# Step 3: Predict on testData and Compute the confusion matrix
predicted3 <- predict(model_mars3, testData4)
confusionMatrix(reference = testData$Purchase, data = predicted3, mode='everything', positive='MM')


# 8. How to evaluate performance of multiple machine learning algorithms?

Caret provides the resamples() function where you can provide multiple machine learning models and collectively evaluate them.

Let’s first train some more algorithms.

# 8.1. Training Adaboost

In [ ]:
set.seed(100)

# Train the model using adaboost
model_adaboost = train(Purchase ~ ., data=trainData, method='adaboost', tuneLength=2, trControl = fitControl)
model_adaboost

# 8.2. Training Random Forest

In [ ]:
set.seed(100)

# Train the model using rf
model_rf = train(Purchase ~ ., data=trainData, method='rf', tuneLength=5, trControl = fitControl)
model_rf

# 8.3. Training xgBoost Dart

In [ ]:
set.seed(100)

# Train the model using MARS
model_xgbDART = train(Purchase ~ ., data=trainData, method='xgbDART', tuneLength=5, trControl = fitControl, verbose=F)
model_xgbDART

# 8.4. Training SVM

In [ ]:
set.seed(100)

# Train the model using MARS
model_svmRadial = train(Purchase ~ ., data=trainData, method='svmRadial', tuneLength=15, trControl = fitControl)
model_svmRadial

# 8.5. Run resamples() to compare the models

In [ ]:
# Compare model performances using resample()
models_compare <- resamples(list(ADABOOST=model_adaboost, RF=model_rf, XGBDART=model_xgbDART, MARS=model_mars3, SVM=model_svmRadial))

# Summary of the models performances
summary(models_compare)